In [1]:
import numpy as np
import re
import spacy
from smart_open import smart_open
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Flatten
from keras.layers import Conv1D, MaxPooling1D, LSTM, Embedding, Dropout


Using TensorFlow backend.
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflo

In [11]:
import pandas as pd
X = pd.read_csv('data_clean/vectorize_text .csv', header = None)
y = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]


In [6]:
display(X)

,0,1,2,3,4,5,6,7,8,9,...,201770,201771,201772,201773,201774,201775,201776,201777,201778,201779
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,361.0,3110.0,15.0,49.0,207.0,4.0,174.0,2.0,54.0,8404.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,1315.0,2.0,229.0,3.0,907.0,218.0,1.0,1681.0,567.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,75.0,2.0,1287.0,1556.0,176.0,1622.0,4.0,1.0,206.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6186.0,18.0,9.0,42.0,631.0,116.0,16.0,4822.0,3.0,1198.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,30.0,35.0,1057.0,4.0,164.0,144.0,54.0,81.0,1083.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,25.0,292.0,3.0,1062.0,1.0,97.0,525.0,3.0,167.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,529.0,30.0,18.0,8.0,156.0,449.0,7.0,116.0,858.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8181.0,2.0,6.0,391.0,463.0,10.0,15.0,25.0,67.0,2661.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2433.0,2.0,133.0,960.0,25.0,1449.0,1.0,156.0,40.0,630.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,7.0,4.0,266.0,231.0,8.0,15.0,3108.0,4090.0,11262.0


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 201780 entries, 0 to 201779
dtypes: float64(201780)
memory usage: 30.8 MB


In [12]:
model = Sequential()
model.add(Embedding(50000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 201780, 256)       12800000  
_________________________________________________________________
dropout_2 (Dropout)          (None, 201780, 256)       0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 201780, 256)       525312    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 13,851,909
Trainable params: 13,851,909
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs= 8, batch_size= 32, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/8
